In [15]:
import tensorflow as tf
from tensorflow import data
import shutil
import math
from datetime import datetime
from tensorflow.python.feature_column import feature_column

from tensorflow.contrib.learn import learn_runner
from tensorflow.contrib.learn import make_export_strategy

print(tf.__version__)

1.4.0


## Steps to use the TF Estimator APIs
1. Define dataset **metadata**
2. Define **data input function** to read the data from the source + **apply pre-processing**
3. Create TF **feature columns** based on metadata + **extended feature columns**
4. Instantiate a **model function** with the required **feature columns, EstimatorSpecs, & parameters**
5. Define a **serving function**
6. Run **Experiment** by supplying training and validation data, as well as required parameters
7. **Evaluate** the model using test data
8. Perform **predictions**

In [2]:
train_data_files = ['data/train-data.csv']
valid_data_files = ['data/valid-data.csv']
test_data_files = ['data/test-data.csv']

model_name = 'reg-model-04'

resume = False
train = True
preprocess = True

## 1. Define Dataset Metadata

In [3]:
HEADER = ['key','x','y','alpha','beta','target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]

NUMERIC_FEATURE_NAMES = ['x', 'y']  

CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

TARGET_NAME = 'target'

UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})

print("Header: {}".format(HEADER))
print("Numeric Features: {}".format(NUMERIC_FEATURE_NAMES))
print("Categorical Features: {}".format(CATEGORICAL_FEATURE_NAMES))
print("Target: {}".format(TARGET_NAME))
print("Unused Features: {}".format(UNUSED_FEATURE_NAMES))

Header: ['key', 'x', 'y', 'alpha', 'beta', 'target']
Numeric Features: ['x', 'y']
Categorical Features: ['alpha', 'beta']
Target: target
Unused Features: ['key']


## 2. Define Data Input Function

### a. parsing and preprocessing logic

In [4]:
def parse_csv_row(csv_row):
    
    columns = tf.decode_csv(csv_row, record_defaults=HEADER_DEFAULTS)
    features = dict(zip(HEADER, columns))
    
    for column in UNUSED_FEATURE_NAMES:
        features.pop(column)
    
    target = features.pop(TARGET_NAME)

    return features, target

def process_features(features):
    
    if preprocess:
        features["x_2"] = tf.square(features['x'])
        features["y_2"] = tf.square(features['y'])
        features["xy"] = tf.multiply(features['x'], features['y']) # features['x'] * features['y']
        features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    
    return features

### b. data pipeline input function

In [5]:
def csv_input_fn(file_names, mode=tf.estimator.ModeKeys.EVAL, 
                 skip_header_lines=0, 
                 num_epochs=None, 
                 batch_size=200):
    
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    
    print("")
    print("* data input_fn:")
    print("================")
    print("Input file(s): {}".format(file_names))
    print("Batch size: {}".format(batch_size))
    print("Epoch Count: {}".format(num_epochs))
    print("Mode: {}".format(mode))
    print("Shuffle: {}".format(shuffle))
    print("================")
    print("")

    dataset = data.TextLineDataset(filenames=file_names)
    dataset = dataset.skip(skip_header_lines)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda csv_row: parse_csv_row(csv_row))
    dataset = dataset.map(lambda features, target: (process_features(features), target))
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    
    features, target = iterator.get_next()
    return features, target

In [6]:
features, target = csv_input_fn(file_names=train_data_files)
print("Feature read from CSV: {}".format(list(features.keys())))
print("Target read from CSV: {}".format(target))


* data input_fn:
Input file(s): ['data/train-data.csv']
Batch size: 200
Epoch Count: None
Mode: eval
Shuffle: False

Feature read from CSV: ['x', 'y', 'alpha', 'beta', 'x_2', 'y_2', 'xy', 'dist_xy']
Target read from CSV: Tensor("IteratorGetNext:8", shape=(?,), dtype=float32)


## 3. Define Feature Columns

In [7]:
def get_feature_columns():
    
    CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']
    all_numeric_feature_names = NUMERIC_FEATURE_NAMES.copy() 
    
    if preprocess:
        all_numeric_feature_names += CONSTRUCTED_NUMERIC_FEATURES_NAMES

    numeric_columns = {feature_name: tf.feature_column.numeric_column(feature_name)
                       for feature_name in all_numeric_feature_names}

    categorical_column_with_vocabulary = \
        {item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
         for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}
        
    feature_columns = {}

    if numeric_columns is not None:
        feature_columns.update(numeric_columns)

    if categorical_column_with_vocabulary is not None:
        feature_columns.update(categorical_column_with_vocabulary)
        
    
    # apply feature extentions
    
    feature_columns['alpha_X_beta'] = tf.feature_column.crossed_column(
        [feature_columns['alpha'], feature_columns['beta']], 4)
    
    return feature_columns

feature_columns = get_feature_columns()
print("Feature Columns: {}".format(feature_columns))

Feature Columns: {'x': _NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y': _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'x_2': _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y_2': _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'xy': _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'dist_xy': _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'alpha': _VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'beta': _VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'alpha_X_beta': _CrossedColumn(keys=(_VocabularyListCategoricalColumn(key=

## 4. Define Model Function

In [8]:
def regression_model_fn(features, labels, mode, params):

    hidden_units = params.hidden_units
    output_layer_size = 1

    feature_columns = list(get_feature_columns().values())

    dense_columns = list(
        filter(lambda column: isinstance(column, feature_column._NumericColumn),
               feature_columns
        )
    )

    categorical_columns = list(
        filter(lambda column: isinstance(column, feature_column._VocabularyListCategoricalColumn) |
                              isinstance(column, feature_column._BucketizedColumn),
                   feature_columns)
    )

    indicator_columns = list(
            map(lambda column: tf.feature_column.indicator_column(column),
                categorical_columns)
    )


    # Create the input layers from the features
    input_layer = tf.feature_column.input_layer(features= features, 
                                                feature_columns= dense_columns+indicator_columns)

#     # Create only 1 hidden layer based on the first element of the hidden_units in the params
#     hidden_layer_size = hidden_units[0]
#     hidden_layer = tf.layers.dense(inputs= input_layer, 
#                                    units=hidden_layer_size, 
#                                    activation=tf.nn.relu)

    # Create a fully-connected layer-stack based on the hidden_units in the params
    hidden_layers = tf.contrib.layers.stack(inputs= input_layer,
                                            layer= tf.contrib.layers.fully_connected,
                                            stack_args= hidden_units)

    # Connect the output layer (logits) to the hidden layer (no activation fn)
    logits = tf.layers.dense(inputs=hidden_layers, 
                             units=output_layer_size)

    # Reshape output layer to 1-dim Tensor to return predictions
    output = tf.squeeze(logits)

    # Provide an estimator spec for `ModeKeys.PREDICT`.
    if mode == tf.estimator.ModeKeys.PREDICT:

        predictions = {
            'scores': output
        }

        export_outputs = {
            'predictions': tf.estimator.export.PredictOutput(predictions)
        }

        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            export_outputs=export_outputs)

    # Calculate loss using mean squared error
    loss = tf.losses.mean_squared_error(labels, output)

    # Create Optimiser
    optimizer = tf.train.AdamOptimizer()

    # Create training operation
    train_op = optimizer.minimize(
        loss=loss, global_step=tf.train.get_global_step())

    # Calculate root mean squared error as additional eval metric
    eval_metric_ops = {
        "rmse": tf.metrics.root_mean_squared_error(
            labels, output)
    }

    # Provide an estimator spec for `ModeKeys.EVAL` and `ModeKeys.TRAIN` modes.
    estimator_spec = tf.estimator.EstimatorSpec(mode=mode,
                                                loss=loss,
                                                train_op=train_op,
                                                eval_metric_ops=eval_metric_ops)
    return estimator_spec


def create_estimator(run_config, hparams):
    estimator = tf.estimator.Estimator(model_fn=regression_model_fn, 
                                  params=hparams, 
                                  config=run_config)
    
    print("")
    print("Estimator Type: {}".format(type(estimator)))
    print("")

    return estimator

## 5. Define Serving Funcion

In [9]:
def csv_serving_input_fn():
    
    SERVING_HEADER = ['x','y','alpha','beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]

    rows_string_tensor = tf.placeholder(dtype=tf.string,
                                         shape=[None],
                                         name='csv_rows')
    
    receiver_tensor = {'csv_rows': rows_string_tensor}

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))

    return tf.estimator.export.ServingInputReceiver(
        process_features(features), receiver_tensor)

## 6. Run Experiment

### a. Define Experiment Function

In [10]:
def generate_experiment_fn(**experiment_args):

    def _experiment_fn(run_config, hparams):

        train_input_fn = lambda: csv_input_fn(
            train_data_files,
            mode = tf.contrib.learn.ModeKeys.TRAIN,
            num_epochs=hparams.num_epochs,
            batch_size=hparams.batch_size
        )

        eval_input_fn = lambda: csv_input_fn(
            valid_data_files,
            mode=tf.contrib.learn.ModeKeys.EVAL,
            num_epochs=1,
            batch_size=hparams.batch_size
        )

        estimator = create_estimator(run_config, hparams)

        return tf.contrib.learn.Experiment(
            estimator,
            train_input_fn=train_input_fn,
            eval_input_fn=eval_input_fn,
            eval_steps=None,
            #min_eval_frequency=1,
            #eval_hooks=[validation_monitor],
            **experiment_args
        )

    return _experiment_fn

### b. Set HParam and RunConfig

In [11]:
hparams  = tf.contrib.training.HParams(
    num_epochs = 1000,
    batch_size = 500,
    hidden_units=[8, 4], 
    dropout_prob = 0.0)

model_dir = 'trained_models/{}'.format(model_name)

run_config = tf.contrib.learn.RunConfig(
    save_checkpoints_steps=2400,
    tf_random_seed=19850610,
    model_dir=model_dir
)

print(run_config.model_dir)

trained_models/reg-model-04


### c. Run Experiment via learn_runner

In [12]:
if not resume:
    print("Removing previous artifacts...")
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print("Resuming training...") 

if train:
    tf.logging.set_verbosity(tf.logging.INFO)
    
    time_start = datetime.utcnow() 
    print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
    print(".......................................") 
    
    learn_runner.run(
        experiment_fn=generate_experiment_fn(

            export_strategies=[make_export_strategy(
                csv_serving_input_fn,
                exports_to_keep=1
            )]
        ),
        run_config=run_config,
        schedule="train_and_evaluate",
        hparams=hparams
    )
    
    time_end = datetime.utcnow() 
    print(".......................................")
    print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
    print("")
    time_elapsed = time_end - time_start
    print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))
    

Removing previous artifacts...
Experiment started at 16:12:22
.......................................
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x127470550>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 19850610, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 2400, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/reg-model-04'}

Estimator Type: <class 'tensorflow.python.estimator.estimator.Estimator'>

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.

* data input_fn:
Input file(s): ['data/train-data.csv']
Batch size: 500
Epoch Cou

INFO:tensorflow:loss = 84.5367, step = 4801 (4.103 sec)
INFO:tensorflow:global_step/sec: 31.5467
INFO:tensorflow:loss = 94.9545, step = 4901 (1.045 sec)
INFO:tensorflow:global_step/sec: 109.145
INFO:tensorflow:loss = 90.2393, step = 5001 (0.917 sec)
INFO:tensorflow:global_step/sec: 111.017
INFO:tensorflow:loss = 115.458, step = 5101 (0.901 sec)
INFO:tensorflow:global_step/sec: 121.401
INFO:tensorflow:loss = 96.7338, step = 5201 (0.824 sec)
INFO:tensorflow:global_step/sec: 110.137
INFO:tensorflow:loss = 89.3789, step = 5301 (0.908 sec)
INFO:tensorflow:global_step/sec: 107.05
INFO:tensorflow:loss = 103.783, step = 5401 (0.935 sec)
INFO:tensorflow:global_step/sec: 114.25
INFO:tensorflow:loss = 111.459, step = 5501 (0.873 sec)
INFO:tensorflow:global_step/sec: 138.917
INFO:tensorflow:loss = 81.5086, step = 5601 (0.720 sec)
INFO:tensorflow:global_step/sec: 132.97
INFO:tensorflow:loss = 101.688, step = 5701 (0.753 sec)
INFO:tensorflow:global_step/sec: 113.356
INFO:tensorflow:loss = 91.7086, s

INFO:tensorflow:Saving checkpoints for 12001 into trained_models/reg-model-04/model.ckpt.
INFO:tensorflow:global_step/sec: 72.4096

* data input_fn:
Input file(s): ['data/valid-data.csv']
Batch size: 500
Epoch Count: 1
Mode: eval
Shuffle: False

INFO:tensorflow:Starting evaluation at 2017-11-03-16:14:08
INFO:tensorflow:Restoring parameters from trained_models/reg-model-04/model.ckpt-12001
INFO:tensorflow:Finished evaluation at 2017-11-03-16:14:08
INFO:tensorflow:Saving dict for global step 12001: global_step = 12001, loss = 101.069, rmse = 10.0533
INFO:tensorflow:Validation (step 12001): loss = 101.069, rmse = 10.0533, global_step = 12001
INFO:tensorflow:loss = 99.8998, step = 12001 (2.820 sec)
INFO:tensorflow:global_step/sec: 47.7248
INFO:tensorflow:loss = 93.8803, step = 12101 (0.657 sec)
INFO:tensorflow:global_step/sec: 152.081
INFO:tensorflow:loss = 79.6604, step = 12201 (0.657 sec)
INFO:tensorflow:global_step/sec: 150.858
INFO:tensorflow:loss = 99.4033, step = 12301 (0.663 sec)
IN

INFO:tensorflow:loss = 82.2627, step = 18401 (0.666 sec)
INFO:tensorflow:global_step/sec: 149.835
INFO:tensorflow:loss = 86.1882, step = 18501 (0.667 sec)
INFO:tensorflow:global_step/sec: 150.349
INFO:tensorflow:loss = 91.3221, step = 18601 (0.665 sec)
INFO:tensorflow:global_step/sec: 148.406
INFO:tensorflow:loss = 87.5002, step = 18701 (0.674 sec)
INFO:tensorflow:global_step/sec: 148.267
INFO:tensorflow:loss = 89.7652, step = 18801 (0.677 sec)
INFO:tensorflow:global_step/sec: 135.322
INFO:tensorflow:loss = 67.1995, step = 18901 (0.736 sec)
INFO:tensorflow:global_step/sec: 153.074
INFO:tensorflow:loss = 62.0152, step = 19001 (0.654 sec)
INFO:tensorflow:global_step/sec: 153.564
INFO:tensorflow:loss = 84.1627, step = 19101 (0.651 sec)
INFO:tensorflow:Saving checkpoints for 19201 into trained_models/reg-model-04/model.ckpt.
INFO:tensorflow:global_step/sec: 83.2586

* data input_fn:
Input file(s): ['data/valid-data.csv']
Batch size: 500
Epoch Count: 1
Mode: eval
Shuffle: False

INFO:tensor

## 7. Evaluate the Model

In [13]:
train_size = 12000
valid_size = 3000
test_size = 5000

train_input_fn = lambda: csv_input_fn(file_names= train_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= train_size)

valid_input_fn = lambda: csv_input_fn(file_names= valid_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= valid_size)

test_input_fn = lambda: csv_input_fn(file_names= test_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= test_size)

estimator = create_estimator(run_config, hparams)

train_results = estimator.evaluate(input_fn=train_input_fn, steps=1)
train_rmse = str(train_results["rmse"])
print()
print("######################################################################################")
print("# Train RMSE: {} - {}".format(train_rmse, train_results))
print("######################################################################################")

valid_results = estimator.evaluate(input_fn=valid_input_fn, steps=1)
valid_rmse = str(valid_results["rmse"])
print()
print("######################################################################################")
print("# Valid RMSE: {} - {}".format(valid_rmse,valid_results))
print("######################################################################################")

test_results = estimator.evaluate(input_fn=test_input_fn, steps=1)
test_rmse = str(test_results["rmse"])
print()
print("######################################################################################")
print("# Test RMSE: {} - {}".format(test_rmse, test_results))
print("######################################################################################")

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x127470550>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 19850610, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 2400, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/reg-model-04'}

Estimator Type: <class 'tensorflow.python.estimator.estimator.Estimator'>


* data input_fn:
Input file(s): ['data/train-data.csv']
Batch size: 12000
Epoch Count: None
Mode: eval
Shuffle: False

INFO:tensorflow:Starting evaluation at 2017-11-03-16:15:39
INFO:tensorflow:Restoring parameters from trained_models/reg-model-04/model.ckpt-24000
INFO:t

## 8. Prediction

In [14]:
import itertools

predict_input_fn = lambda: csv_input_fn(file_names= test_data_files, 
                                      mode= tf.estimator.ModeKeys.PREDICT,
                                      batch_size= 5)

predictions = estimator.predict(input_fn=predict_input_fn)
print("")
print(list(itertools.islice(predictions, 5)))



* data input_fn:
Input file(s): ['data/test-data.csv']
Batch size: 5
Epoch Count: None
Mode: infer
Shuffle: False

INFO:tensorflow:Restoring parameters from trained_models/reg-model-04/model.ckpt-24000
[{'scores': 55.24572}, {'scores': -2.8034539}, {'scores': 18.621042}, {'scores': 2.9108515}, {'scores': 1.5959911}]
